In [1]:
# Todas bibliotecas serão testadas utilizando o mesmo arquivo
file_path_com_tabela = "../data/290-licitacoes-cristais/data/files/94cbb7c0710eda204d32940b26503796.pdf"
file_path_sem_tabela = "../data/288_licitacoes-pirapetinga/data/files/caf3d38ce4042bfe4a0534a4d478fbf5"
file_path_escaneado = "../data/306-licitacoes-arantina/data/files/751442f1e09057023a7d3acd1b055bfb.pdf"

file_path = file_path_com_tabela

# Apache tika 
https://tika.apache.org/

- retorna um dicionário com keys ['metadata', 'content', 'status']
    - metadata: não é muito informativo
    - content: texto é bem formatado 
        - é possível extrair informações por linha
        - não é possível extrair por página. Comentário Michele: É possível extrair por páginas convertendo para XML
        - tabelas perdem a formatação. Comentário Michele: Convertendo para XML parece ficar mais fácil de processar. Iago: olhando o html eu fiquei com a impressão de que ainda não daria para estruturar as informações das tabelas. Michele: Também estou na dúvida, pois a estrutura de tabela se perde, mas cada linha fica com informações em sequência. O difícil é saber que é uma tabela, mas com palavras-chave dá para buscar a informação. Entretanto, teria que identificar automaticamente qual linha se refere à descrição das colunas de uma tabela, para então achar as linhas com conteúdo. Pensando nisso, acho que uma lib que mantenha a estrutura da tabela pode ser mais fácil para realizar o processamento automático.
    - status: retorna um código utilizado pela lib, não informativo

In [2]:
from tika import parser
#Bibliotecas para processar informações de cada página
from io import StringIO
from bs4 import BeautifulSoup

# retorna um dicionário
tika_parser = parser.from_file(file_path)
# metada: não é muito informativo
# content: Texto bem formatado, é possível extrair as linhas do texto, informações em tabelas perdem formatação
print(tika_parser.keys())

print(tika_parser['metadata'])

print(tika_parser['content'])

print(tika_parser['status'])


#Verificando processamento por páginas com apache Tika - converte o conteúdo para XML e processa com BeautifulSoup
#Possibilita contar a quantidade de páginas e pegar informações das tabelas
file_data = []
_buffer = StringIO()
data = parser.from_file(file_path, xmlContent=True)
xhtml_data = BeautifulSoup(data['content'])
for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
    print('Parsing page {} of pdf file...'.format(page+1))
    _buffer.write(str(content))
    parsed_content = parser.from_buffer(_buffer.getvalue())
    _buffer.truncate()
    #Saída em formato json
    file_data.append({'id': 'page_'+str(page+1), 'content': parsed_content['content']})
    
print(file_data)

#Outro formato de processamento por páginas
pages_txt = []

data = parser.from_file(file_path, xmlContent=True)
xhtml_data = BeautifulSoup(data['content'])
for i, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
    # Parse PDF data using TIKA (xml/html)
    # É seguro e rápido criar um novo buffer e não truncar este
    # https://stackoverflow.com/questions/4330812/how-do-i-clear-a-stringio-object
    _buffer = StringIO()
    _buffer.write(str(content))
    parsed_content = parser.from_buffer(_buffer.getvalue())

    # Add pages
    text = parsed_content['content'].strip()
    pages_txt.append(text)

print(pages_txt)

dict_keys(['metadata', 'content', 'status'])
{'Author': 'Pablo_PC', 'Content-Type': 'application/pdf', 'Creation-Date': '2017-09-29T14:52:13Z', 'Last-Modified': '2017-09-29T14:52:13Z', 'Last-Save-Date': '2017-09-29T14:52:13Z', 'X-Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'X-TIKA:embedded_depth': '0', 'X-TIKA:parse_time_millis': '66', 'access_permission:assemble_document': 'true', 'access_permission:can_modify': 'true', 'access_permission:can_print': 'true', 'access_permission:can_print_degraded': 'true', 'access_permission:extract_content': 'true', 'access_permission:extract_for_accessibility': 'true', 'access_permission:fill_in_form': 'true', 'access_permission:modify_annotations': 'true', 'created': '2017-09-29T14:52:13Z', 'creator': 'Pablo_PC', 'date': '2017-09-29T14:52:13Z', 'dc:creator': 'Pablo_PC', 'dc:format': 'application/pdf; version=1.7', 'dcterms:created': '2017-09-29T14:52:

['ATA DO PREGÃO PRESENCIAL Nº 00058/2017\n\n\n        Aos 27/09/2017 às 09:00horas, na sala de Licitações da Prefeitura Municipal de Cristais, situado na Praça Cel. Joaquim Luiz da Costa Maia,\n001,  Centro,  em  Cristais/MG,  realizou-se  sessão  pública  para  recebimento  e  abertura  dos  envelopes,  contendo  as  propostas  de  preços  e  a\ndocumentação  de  habilitação,  apresentados  em  razão  do  certame  licitatório  na  modalidade  de  Pregão  Presencial  em  referência,  cujo  objeto  é:\nAQUISIÇÕES  FUTURAS  E  INCERTAS  DE  BRINQUEDOS  PEDAGÓGICOS  PARA  ATENDER  ÀS  NECESSIDADES  DA  DIRETORIA\nMUNICIPAL DE EDUCAÇÃO DO MUNICÍPIO DE CRISTAIS Presentes o pregoeiro Mattheus Henrique Rogana, bem como a equipe de apoio:\n\n\nEquipe de Apoio\n\n\nEDINAMARA RIBEIRO SILVA, PABLO JOSE DA SILVA\n\n\n,  que  esta  subscrevem designados  pela  Portaria  nº  092/17 -  21/08/2017,  para  atendimento  às  disposições  contidas  no  Decreto  nº  031/2009  de\n10/06/2009. Presentes tamb

# PDFMiner
https://pdfminersix.readthedocs.io/en/latest/

- Mal documentada, de uso complexo
- O texto extraído mantém uma certa formatação possibilitando extração por linha
- Tabelas perdem formatação
- Permite processamento por páginas
- As informações dos metadados não parecem muito relevantes. Por exemplo: [{'Author': b'Pablo_PC', 'CreationDate': b"D:20170929115213-03'00'", 'Creator': b'PDF Architect 4', 'ModDate': b"D:20170929115213-03'00'", 'Producer': b'PDF Architect 4'}]


In [3]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

#Imports para capturar metadados
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text
print(convert_pdf_to_txt(file_path))

#Metadados com PDFMiner
fp = open(file_path, 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)

print(">> Metadados: ")
print(doc.info)  # The "Info" metadata

ATA DO PREGÃO PRESENCIAL Nº 00058/2017

        Aos 27/09/2017 às 09:00horas, na sala de Licitações da Prefeitura Municipal de Cristais, situado na Praça Cel. Joaquim Luiz da Costa Maia,
001,  Centro,  em  Cristais/MG,  realizou-se  sessão  pública  para  recebimento  e  abertura  dos  envelopes,  contendo  as  propostas  de  preços  e  a
documentação  de  habilitação,  apresentados  em  razão  do  certame  licitatório  na  modalidade  de  Pregão  Presencial  em  referência,  cujo  objeto  é:
AQUISIÇÕES  FUTURAS  E  INCERTAS  DE  BRINQUEDOS  PEDAGÓGICOS  PARA  ATENDER  ÀS  NECESSIDADES  DA  DIRETORIA
MUNICIPAL DE EDUCAÇÃO DO MUNICÍPIO DE CRISTAIS Presentes o pregoeiro Mattheus Henrique Rogana, bem como a equipe de apoio:

Equipe de Apoio

EDINAMARA RIBEIRO SILVA, PABLO JOSE DA SILVA
,  que  esta  subscrevem designados  pela  Portaria  nº  092/17  -  21/08/2017,  para  atendimento  às  disposições  contidas  no  Decreto  nº  031/2009  de
10/06/2009. Presentes também, os representantes d

In [4]:
from pdfminer.pdfinterp import PDFResourceManager 
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import HTMLConverter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from io import BytesIO
import re


def convert_pdf_to_html(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 3 #is for all
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str
# Se copiarmos esta saída e colarmos em um arquivo html, da pra ver que ele consegue manter 
# um pouco a estrutura do documento. Isso pode ajudar no processo de extração de tabelas dos 
# documentos. Porém, é mantida apenas uma relação espacial, baseada na posição das informações.
# Essa abordagem é interessante por através do tamanho da fonte, permitir a extração do título do documento.
print(convert_pdf_to_html(file_path))

b'<html><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n</head><body>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:595px; height:841px;"></span>\n<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>\n<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:176px; top:176px; width:256px; height:12px;"><span style="font-family: TimesNewRomanPS-BoldMT; font-size:12px">ATA DO PREG\xc3\x83O PRESENCIAL N\xc2\xba 00058/2017\n<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:32px; top:218px; width:545px; height:55px;"><span style="font-family: TimesNewRomanPSMT; font-size:12px">       </span><span style="font-family: TimesNewRomanPSMT; font-size:9px"> Aos 27/09/2017 \xc3\xa0s 09:00horas, na sala de Licita\xc3\xa7\xc3\xb5es da Prefeitura Municipal de Cristais, situado na Pra\xc3\xa7a Cel. Joaquim Luiz da Costa Maia,\n<br>001,  Centro,  

# Tabula
https://tabula-py.readthedocs.io/en/latest/
- funciona muito bem para arquivos que possuem apenas tabelas
- flag stream permite focar extração em informações com formato de tabela, ignorando texto
    - Algumas tabelas acabam sendo ignoradas
    - Pulo de linha são identificados como NaN
- Permite facilmente controlar a quantidade de páginas a serem processadas

In [4]:
import tabula

tabula_parser = tabula.read_pdf(
    file_path,
    pages="1,2",
    multiple_tables=True,
    stream=True # importante para que sejam extraídos apenas onde tem estrutura parecida com tabela
)

print(tabula_parser)

[                                     Unnamed: 0  Equipe de Apoio
0  EDINAMARA RIBEIRO SILVA, PABLO JOSE DA SILVA              NaN,                                         LICITANTE(S)  \
0                                 11.384.015/0001-17   
1  ATLANTICA DIDÁTICA & PEDAGOGIA DISTRIBUIDORA L...   
2                                 21.613.635/0001-34   
3                  CARLOS ROBERTO DA SILVA REIS - ME   
4                                 02.328.587/0001-98   
5  MARCELO PEREIRA DOS SANTOS - CPF 74263242904 - ME   
6                                 13.319.237/0001-81   
7                             SHOP MINEIRO LTDA - ME   

                 REPRESENTANTE(S)  
0             CPF: 625.083.696-91  
1     WANDERLEY GONÇALVES FONSECA  
2             CPF: 887.147.566-68  
3  ALECXANDRO DE OLIVEIRA PORFÍRI  
4             CPF: 742.632.429-04  
5      MARCELO PEREIRA DOS SANTOS  
6             CPF: 904.553.806-78  
7          DONIZETI GERALDO GODOI  ,       11          BAÚ PEDAGÓGICO LIGA 

In [6]:
if len(tabula_parser) != 0:
    tabula_parser[0]

# Slate
https://github.com/timClicks/slate
- Apresenta erro na hora de instalar a lib
- Projeto não é atualizado desde 2017

# PDFquery
https://github.com/jcushman/pdfquery/blob/master/README.rst
- Permite fazer buscas no xml gerado a partir do arquivo
- Separa o arquivo em linhas
- Acaba sendo inviável de extrair as informações de forma genérica
- Última atualização em 2019

In [5]:
import pdfquery

pdf = pdfquery.PDFQuery(file_path)
pdf.load()

In [7]:
label = pdf.pq('LTTextLineHorizontal:contains("REPRESENTANTE(S) ")')

left_corner = float(label.attr('x0'))
bottom_corner = float(label.attr('y0'))
bbox = [float(x) for x in label.attr('bbox').replace("[","").replace("]","").split(",")]
name = pdf.pq('LTTextLineHorizontal:in_bbox("%s, %s, %s, %s")' % (bbox[0], bbox[1], bbox[2], bbox[3])).text()
name



'REPRESENTANTE(S)'

# PyPDF2 
- Não funciona para todos os documentos
- PyPDF2 é uma versão não atualizado, uma nova versão compatível com Python3 é o PyPDF4. Fonte: https://realpython.com/pdf-python/#history-of-pypdf-pypdf2-and-pypdf4
- Última atualização do pyPDF para Python3 foi em 2009: https://github.com/mfenniak/pyPdf/tree/py3


In [9]:
from PyPDF2 import PdfFileReader

In [10]:
pdf = PdfFileReader(file_path)
pdf.getPage(0).extractText()

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

# PyMuPDF
https://github.com/pymupdf/PyMuPDF

- Permite extrair número de páginas
- Permite extrair texto por páginas
- Permite extrair texto em blocos
- Permite fácil busca por termos - Iago: a ferramenta não aceita regex, a busca pelo termo pode acabar trazendo "lixo". Michele: Serve apenas para busca simples, mas acredito que não aceitar regex, é um aspecto importante para não utilizá-la. Apesar de que o texto poderia ser armazenado em um dataframe e depois utilizado regex para buscar nesse dataframe.

In [34]:
from operator import itemgetter
from itertools import groupby

import fitz

doc = fitz.open( file_path_sem_tabela)
print("Numero de paginas: ", doc.pageCount)
doc.metadata # metadata
pdf = doc[0] # Seleciona primeira pagina
print(pdf.get_text()) # Texto da primeira pagina

#Busca por termos
search_term = "ata"
for current_page in range(len(doc)):
    page = doc.loadPage(current_page)
    if page.searchFor(search_term):
        print("%s found on page %i" % (search_term, current_page))

Numero de paginas:  3
PREFEITURA MUNICIPAL DE PIRAPETINGA 
CEP.: 36.730-000 – ESTADO DE MINAS GERAIS 
 
PRAÇA DIRCEU DE OLIVEIRA MARTINS, 01. 
Tel.: (32) 3465 – 3100 – FAX (32) 3465 – 3101 – CNPJ.: 18.092.825/0001-49 
e-mail.: admpmp@pirapetinga.mg.gov.br 
 
7º TERMO ADITIVO AO CONTRATO N.º 073/2018 DO PROCESSO LICITATÓRIO Nº 
082/2018, 
CELEBRADO 
PELO 
MUNICÍPIO 
DE 
PIRAPETINGA/MG 
E 
MAFEL 
CONSTRUTORA DE PADUA LTDA –ME, QUE TEM POR OBJETO CONTRATAÇÃO DE 
EMPRESA  ESPECIALIZADA  PARA  EXECUÇÃO  DE  MÃO DE OBRA. 
 
CONTRATANTE: MUNICÍPIO DE PIRAPETINGA, inscrito no CNPJ 18.092.825/0001-49, 
neste ato representado pelo Prefeito Municipal Sr. Enoghalliton de Abreu Arruda, brasileiro, 
solteiro, residente e domiciliado no Município de Pirapetinga, doravante denominado simplesmente 
CONTRATANTE. 
 
EMPRESA CONTRATADA: MAFEL CONSTRUTORA DE PADUA LTDA –ME inscrito(a) no 
CNPJ/MF sob o nº 20.157.057/0001-06, sediado(a) na Rua Guilherme Tavares da Silva, 65, Bairro 
Tavares, em Santo Antôni

In [35]:
# Extrai o texto por blocos
# Pode ser mais útil que utilizando o texto inteiro de uma vez
blocks = pdf.getText("blocks")
blocks.sort(key=lambda block: block[1])  # sort vertically ascending

for b in blocks:
    print(b[4]) 
    print("----------------------------------------------------")

<image: DeviceRGB, width: 233, height: 232, bpc: 8>
----------------------------------------------------
PREFEITURA MUNICIPAL DE PIRAPETINGA 

----------------------------------------------------
CEP.: 36.730-000 – ESTADO DE MINAS GERAIS 

----------------------------------------------------
 

----------------------------------------------------
 

----------------------------------------------------
7º TERMO ADITIVO AO CONTRATO N.º 073/2018 DO PROCESSO LICITATÓRIO Nº 
082/2018, 
CELEBRADO 
PELO 
MUNICÍPIO 
DE 
PIRAPETINGA/MG 
E 
MAFEL 

----------------------------------------------------
CONSTRUTORA DE PADUA LTDA –ME, QUE TEM POR OBJETO CONTRATAÇÃO DE 
EMPRESA  ESPECIALIZADA  PARA  EXECUÇÃO  DE  MÃO DE OBRA. 

----------------------------------------------------
 

----------------------------------------------------
<image: DeviceRGB, width: 391, height: 393, bpc: 8>
----------------------------------------------------
CONTRATANTE: MUNICÍPIO DE PIRAPETINGA, inscrito no CNPJ 18.092.

# pdfplumber
https://github.com/jsvine/pdfplumber

- Permite a extração de informações por páginas
- Boa documentação
- Tem o método extract_tables() que permite extrair informações de tabelas. Mais informações para extrair tabelas de acordo com as configurações delas: https://github.com/jsvine/pdfplumber#extracting-tables e https://github.com/jsvine/pdfplumber/blob/stable/examples/notebooks/extract-table-nics.ipynb

In [8]:
import pdfplumber

with pdfplumber.open(file_path_com_tabela) as pdf:
    first_page = pdf.pages[0]
    print(first_page.extract_text())

ATA DO PREGÃO PRESENCIAL Nº 00058/2017
        Aos 27/09/2017 às 09:00horas, na sala de Licitações da Prefeitura Municipal de Cristais, situado na Praça Cel. Joaquim Luiz da Costa Maia,
001, Centro, em Cristais/MG, realizou-se sessão pública para recebimento e abertura dos envelopes, contendo as propostas de preços e a
documentação de habilitação, apresentados em razão do certame licitatório na modalidade de Pregão Presencial em referência, cujo objeto é:
AQUISIÇÕES  FUTURAS  E  INCERTAS  DE  BRINQUEDOS  PEDAGÓGICOS  PARA ATENDER  ÀS  NECESSIDADES  DA  DIRETORIA
MUNICIPAL DE EDUCAÇÃO DO MUNICÍPIO DE CRISTAIS Presentes o pregoeiro Mattheus Henrique Rogana, bem como a equipe de apoio:
Equipe de Apoio
EDINAMARA RIBEIRO SILVA, PABLO JOSE DA SILVA
, que esta subscrevem designados pela Portaria nº 092/17 - 21/08/2017, para atendimento às disposições contidas no Decreto nº 031/2009 de
10/06/2009. Presentes também, os representantes das empresas licitantes, conforme relação abaixo:
LICITANTE(S

In [32]:
import pdfplumber

with pdfplumber.open(file_path_sem_tabela) as pdf:
    first_page = pdf.pages[0]
    print(first_page.extract_text())

PREFEITURA MUNICIPAL DE PIRAPETINGA 
CEP.: 36.730-000 – ESTADO DE MINAS GERAIS 
 
 
7º  TERMO  ADITIVO  AO  CONTRATO  N.º  073/2018  DO  PROCESSO  LICITATÓRIO  Nº 
082/2018,  CELEBRADO  PELO  MUNICÍPIO  DE  PIRAPETINGA/MG  E  MAFEL 
CONSTRUTORA DE PADUA LTDA –ME, QUE TEM POR OBJETO CONTRATAÇÃO DE 
EMPRESA  ESPECIALIZADA  PARA  EXECUÇÃO  DE  MÃO DE OBRA. 
 
CONTRATANTE:  MUNICÍPIO  DE  PIRAPETINGA,  inscrito  no  CNPJ  18.092.825/0001-49, 
neste  ato  representado  pelo  Prefeito  Municipal  Sr.  Enoghalliton  de  Abreu  Arruda,  brasileiro, 
solteiro, residente e domiciliado no Município de Pirapetinga, doravante denominado simplesmente 
CONTRATANTE. 
 
EMPRESA CONTRATADA: MAFEL CONSTRUTORA DE PADUA LTDA –ME inscrito(a) no 
CNPJ/MF sob o nº 20.157.057/0001-06, sediado(a) na Rua Guilherme Tavares da Silva, 65, Bairro 
Tavares, em Santo Antônio de Pádua, Estado do Rio de Janeiro, neste ato representada pelo Sr. 
Felipe Chaves Rodrigues Silva, portador da carteira de identidade nº 21.425.

In [33]:
import pdfplumber

with pdfplumber.open(file_path_escaneado) as pdf:
    first_page = pdf.pages[0]
    print(first_page.extract_text())

None


In [14]:
# Extracao de tabelas
for table in first_page.extract_tables():
    print(table)
    print()

[['Equipe de Apoio'], ['EDINAMARA RIBEIRO SILVA, PABLO JOSE DA SILVA']]

[['LICITANTE(S)', 'REPRESENTANTE(S)'], ['11.384.015/0001-17\nATLANTICA DIDÁTICA & PEDAGOGIA DISTRIBUIDORA LTDA - EPP', 'CPF: 625.083.696-91\nWANDERLEY GONÇALVES FONSECA'], ['21.613.635/0001-34\nCARLOS ROBERTO DA SILVA REIS - ME', 'CPF: 887.147.566-68\nALECXANDRO DE OLIVEIRA PORFÍRI'], ['02.328.587/0001-98\nMARCELO PEREIRA DOS SANTOS - CPF 74263242904 - ME', 'CPF: 742.632.429-04\nMARCELO PEREIRA DOS SANTOS'], ['13.319.237/0001-81\nSHOP MINEIRO LTDA - ME', 'CPF: 904.553.806-78\nDONIZETI GERALDO GODOI']]

[['MARCELO PEREIRA DOS SANTOS - CPF 74263242904 - ME', None, None, None, None, None, None], ['Item', 'Descrição', 'Marca', 'UN', 'Qtd.', 'Valor Unitário', 'Valor Total'], ['', None, None, None, None, None, None], ['1', 'ÁBACO ABERTO', 'ECOBRINK/ECO\nBRINK', 'UN', '10', '19,6000', '196,00'], ['2', 'ALINHAVO DE ANIMAIS', 'UAI\nBRINQUEDOS/U\nAI\nBRINQUEDOS', 'KIT', '30', '35,1000', '1053,00'], ['3', 'ALINHAVO DE NÚMERO